In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, roc_auc_score,accuracy_score,make_scorer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score
import eli5
import ast
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_selection import RFE, RFECV
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
piRNA_rf=pd.read_csv("R/GEO_count/pan_train.csv")

In [4]:
X = piRNA_rf.drop(columns=['group'])
y=piRNA_rf['group']

In [5]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
cv=StratifiedKFold(n_splits=5)
selector = RFECV(estimator=rf_classifier,
                 min_features_to_select=5, 
                 step=1,
                 cv=5,
                 importance_getter='auto',
                 n_jobs=20)

In [ ]:
def evaluate_model(X, y, model, cv, n_resamples=5):
    scores = []
    for i in range(n_resamples):
        # Apply RF-RFE
        model.fit(X, y)
        # Get selected features
        X_selected = X.iloc[:, model.support_]
        # Evaluate model with cross-validation
        score = cross_val_score(model.estimator_, X_selected, y, scoring='accuracy', cv=cv).mean()
        scores.append(score)
    return np.array(scores)

In [ ]:
scores = evaluate_model(X, y, selector, cv, n_resamples=5)
print("Scores from each resampling: ", scores)
print("Mean Score: ", scores.mean())

In [ ]:
print("Optimal number of features: %d" % selector.n_features_)

In [ ]:
selector.cv_results_

In [ ]:
selected_features = [feature for mask, feature in zip(selector.support_, feature_names) if mask]
print("Selected features:", selected_features)

In [ ]:
min_features_to_select=5


n_scores = len(selector.cv_results_["mean_test_score"])
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Mean test accuracy")
plt.errorbar(
    range(min_features_to_select, n_scores + min_features_to_select),
    selector.cv_results_["mean_test_score"],
    yerr=selector.cv_results_["std_test_score"],
)
plt.title("Recursive Feature Elimination \nwith correlated features")
plt.show()